In [43]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Embedding
from keras.models import Model
import numpy as np
import pandas as pd
import string
import re
from string import digits
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

batch_size = 128
epochs = 100
latent_dim = 300
#filename = 'English_hindi.csv'

In [44]:
lines = pd.read_csv("https://raw.githubusercontent.com/prismspeechproject/neural/master/implementation/English_hindi.csv" , encoding='utf-8')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:

lines = lines[lines.columns[0:2]]
# print(lines.isnull().sum(), '\n' , '*' * 20)
# lines.dropna(how='any', inplace=True)
# print(lines.isnull().sum(), '\n' , '*' * 20)
# lines.shape

# OR 

lines=lines[~pd.isnull(lines['English'])]
lines.shape

(127605, 2)

In [46]:
lines.drop_duplicates(inplace=True)
print(lines.shape)
lines.head()

(124818, 2)


,English,Hindi
0,politicians do not have permission to do what ...,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर..."
1,"I'd like to tell you about one such child,",मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2,This percentage is even greater than the perce...,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,what we really mean is that they're bad at not...,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,.The ending portion of these Vedas is called U...,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।


In [47]:
lines=lines.sample(n=25000,random_state=42)
print(lines.shape)
print(lines.columns)
print(lines['English'][0])
lines.head()

# input_texts = []
# target_texts = []
# input_characters = []
# target_characters = []
# target_texts_1 = []
# input_texts_1 = []

# input_texts_1 = lines['English'].values.tolist()
# target_texts_1 = lines['Hindi'].values.tolist()
# a = pd.isnull(input_texts_1)
# b  = pd.isnull(target_texts_1)
# for i in range(len(target_texts_1)):
#   if a[i] == False and b[i] == False:
#     input_texts.append(input_texts_1[i])
#     target_texts.append(target_texts_1[i])
#   else:
#     print(a[i], i)
#     print(input_texts_1[i] , target_texts_1[i])
# print('*' * 10)
# # b  = pd.isnull(target_texts)
# # for i in range(len(b)):
# #   if b[i] == True:
# #     print(b[i], i)
# #     print(target_texts[i])
#     # target_texts.pop(i)
#     # input_texts.pop(i)
# print('*' * 10)
# print(len(input_texts_1))
# print(len(target_texts))

(25000, 2)
Index(['English', 'Hindi'], dtype='object')
politicians do not have permission to do what needs to be done.


,English,Hindi
32537,"to change practical things,","या व्यावहारिक चीज़ों को बदलना भी नहीं,"
67535,Unless the year and all the achievement it con...,अगर यह साल और उसकी उपलइधयां पर्वत के नुकीले शि...
28019,Black-head is a serious disease of the turkeys .,ब्लैकहैड टर्कियों की चिन्ताजनक बीमारी है .
118562,United States of America: 1.790,संयुक्त राज्य अमेरिका : २.७९०
72503,But today other issues have also assumed promi...,लेकिन आज बहुत-से और मसले भी प्रमुख हो गये हैं....


In [48]:
punctuations = set(string.punctuation)
digits_rm = str.maketrans('', '', digits)
# for i in range(len(lines)):
#   lines['English'][i] = re.sub(r"[\(\[].*?[\)\]]", '', lines['English'][i])
#   lines['Hindi'][i] = re.sub(r"[\(\[].*?[\)\]]", "", lines['Hindi'][i])
#   for ch in lines['English'][i]:
#     if ch not in punctuations:
#       lines['English'][i] = ''.join(ch)
#   for ch in lines['Hindi'][i]:
#     if ch not in punctuations:
#       lines['Hindi'][i] = ''.join(ch)
#   lines['English'][i].lower()
#   lines['Hindi'][i].lower()
#   lines['Hindi'][i] = re.sub("[a-z0-9२३०८१५७९४६]", '', lines['Hindi'][i])
#   lines['English'][i] = re.sub("[0-9]", '', lines['English'][i])
#   lines['English'][i] = lines['English'][i].strip()
#   lines['Hindi'][i] = lines['Hindi'][i].strip()

# typing to str
lines['English'] = lines['English'].apply(lambda e: str(e))
lines['Hindi'] = lines['Hindi'].apply(lambda h: str(h))

#Here all the data inside () or [] are removed
lines['English'] = lines['English'].apply(lambda e: re.sub(r"[\(\[].*?[\)\]]", '', e))
lines['Hindi'] = lines['Hindi'].apply(lambda h : re.sub(r"[\(\[].*?[\)\]]", '', h))

#removing punctuations 
lines['English'] = lines['English'].apply(lambda e: ''.join(c for c in e if c not in punctuations))
lines['Hindi'] = lines['Hindi'].apply(lambda h: ''.join(c for c in h if c not in punctuations))

#lower
lines['English'] = lines['English'].apply(lambda e: e.lower())
lines['Hindi'] = lines['Hindi'].apply(lambda h: h.lower())

#Remove digits
lines['English']=lines['English'].apply(lambda e: e.translate(digits_rm))
lines['Hindi']=lines['Hindi'].apply(lambda h: h.translate(digits_rm))
lines['Hindi']=lines['Hindi'].apply(lambda h: re.sub("[a-z२३०८१५७९४६]", '', h))

#Striping
lines['English'] = lines['English'].apply(lambda e: e.strip())
lines['Hindi'] = lines['Hindi'].apply(lambda h: h.strip())

lines.head()

,English,Hindi
32537,to change practical things,या व्यावहारिक चीज़ों को बदलना भी नहीं
67535,unless the year and all the achievement it con...,अगर यह साल और उसकी उपलइधयां पर्वत के नुकीले शि...
28019,blackhead is a serious disease of the turkeys,ब्लैकहैड टर्कियों की चिन्ताजनक बीमारी है
118562,united states of america,संयुक्त राज्य अमेरिका
72503,but today other issues have also assumed promi...,लेकिन आज बहुतसे और मसले भी प्रमुख हो गये हैंये...


In [49]:
#Appending Staring and Ending notation in Hindi column
lines['Hindi'] = lines['Hindi'].apply(lambda h: 'START_ ' + h + ' _END')
lines.head()

,English,Hindi
32537,to change practical things,START_ या व्यावहारिक चीज़ों को बदलना भी नहीं _END
67535,unless the year and all the achievement it con...,START_ अगर यह साल और उसकी उपलइधयां पर्वत के नु...
28019,blackhead is a serious disease of the turkeys,START_ ब्लैकहैड टर्कियों की चिन्ताजनक बीमारी ह...
118562,united states of america,START_ संयुक्त राज्य अमेरिका _END
72503,but today other issues have also assumed promi...,START_ लेकिन आज बहुतसे और मसले भी प्रमुख हो गय...


In [50]:
# extract words
eng_words = []
for sen in lines['English']:
    for w in sen.split():
        if w not in eng_words:
            eng_words.append(w)
hin_words = []
for sen in lines['Hindi']:
    for w in sen.split():
       if w not in hin_words:
          hin_words.append(w)
len(eng_words), len(hin_words)

(30163, 34117)

In [51]:
# Calculate number of words in each line of 2 columns
lines['len_eng_words'] = lines['English'].apply(lambda e: len(e.split(" ")))
lines['len_hin_words'] = lines['Hindi'].apply(lambda h: len(h.split(" ")))
lines.head()

,English,Hindi,len_eng_words,len_hin_words
32537,to change practical things,START_ या व्यावहारिक चीज़ों को बदलना भी नहीं _END,4,9
67535,unless the year and all the achievement it con...,START_ अगर यह साल और उसकी उपलइधयां पर्वत के नु...,41,34
28019,blackhead is a serious disease of the turkeys,START_ ब्लैकहैड टर्कियों की चिन्ताजनक बीमारी ह...,8,8
118562,united states of america,START_ संयुक्त राज्य अमेरिका _END,4,5
72503,but today other issues have also assumed promi...,START_ लेकिन आज बहुतसे और मसले भी प्रमुख हो गय...,11,17


In [52]:
a = lines['len_eng_words'].values.tolist()
#max(lines['len_eng_words']), max(lines['len_hin_words'])
lines[lines['len_eng_words'] > 200]


,English,Hindi,len_eng_words,len_hin_words
83756,the united states department of defense is th...,START_ के द्वारा एक का अध्ययन संयुक्त राज्य अ...,212,160
114935,the idea of adding text messaging to the servi...,START_ मोबाइल फोन ग्राहकों प्रति निवासि मोबाइ...,288,103
75123,missionery position is the most popular and tr...,START_ मिशनरी पोजीशन सर्वाधिक प्रचलित और पारंप...,302,300


In [53]:
lines = lines[lines['len_eng_words']<=20]
lines = lines[lines['len_hin_words']<=20]
max(lines['len_eng_words']), max(lines['len_hin_words'])

(20, 20)

In [54]:
max_len_src = max(lines['len_eng_words'])
max_len_tar = max(lines['len_hin_words'])

In [55]:
input_words = sorted(list(eng_words))
target_words = sorted(list(hin_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
num_decoder_tokens += 1 
num_encoder_tokens = num_encoder_tokens + 1 
num_encoder_tokens , num_decoder_tokens

(30164, 34118)

In [56]:
input_token_index = dict([(word, i+1) for i,word in enumerate(eng_words)])
target_token_index = dict([(word, i+1) for i,word in enumerate(hin_words)])

In [57]:
reverse_input_token_index = dict([(i, word) for word, i in input_token_index.items()])
reverse_target_token_index = dict([(i, word) for word , i in target_token_index.items()])
#reverse_input_token_index, reverse_target_token_index

In [58]:
lines = shuffle(lines)
lines.head()

,English,Hindi,len_eng_words,len_hin_words
115845,so what are we doing with that design,START_ तो हम उस डिजाइन के साथ क्या कर रहे हैं ...,8,12
87420,i took more photos and we started the project,START_ मैंने और तसवीरें खींचीं और हमने प्रोजेक...,9,12
22057,situation,START_ सन्दर्भ _END,1,3
25536,who was a cement hauler,START_ जो कि सीमेंट पीसने का काम करते थे _END,5,10
70644,x glossy photographs of themselves,START_ इनके पास फैंक दी जिसे हम दोस्ती का हाथ ...,6,15


In [59]:
X, Y = lines['English'], lines['Hindi']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2 , random_state=42)
X_train.shape, X_test.shape

((13017,), (3255,))

In [60]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [61]:
def generate_batch(X = X_train, y = Y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_len_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_len_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [62]:

#latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [63]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [64]:
# Create a new model instance
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Save the weights using the `checkpoint_path` format
"""model.save_weights(checkpoint_path.format(epoch=0))"""
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

model.fit_generator(generator = generate_batch(X_train, Y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, Y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    9049200     input_9[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 300)    10235400    input_10[0][0]                   
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
101/101 [==============================] - 53s 439ms/step - loss: 3.9507 - val_loss: 3.3209
Epoch 2/100
101/101 [==============================] - 42s 412ms/step - loss: 3.3591 - val_loss: 3.2548
Epoch 3/100
101/101 [==============================] - 41s 404ms/step - loss: 3.2225 - val_loss: 3.1566
Epoch 4/100
101/101 [==============================] - 41s 407ms/step - loss: 3.0841 - val_loss: 3.0980
Epoch 5/100
101/101 [==============================] - 41s 406ms/step - loss: 2.9664 - val_loss: 3.0664
Epoch 6/100
101/101 [==============================] - 42s 414ms/step - loss: 2.8655 - val_loss: 3.0300
Epoch 7/100
101/101 [==============================] - 42s 411ms/step - loss: 2.7715 - val_loss: 3.0139
Epoch 8/100
101/101 [==============================] - 41s 402ms/step - loss: 2.6847 - val_loss: 2.9949
Epoch 9/100
101/101 [==============================] - 42s 409ms/step - loss: 2.5989 - val_loss: 2.9871
Epoch 10/100
101/101 [==============================] - 41s 405m

In [65]:
#model.save_weights('1stModel.h5')

In [75]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [76]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start word.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [77]:
train_gen = generate_batch(X_test, Y_test, batch_size = 1)
k=-1

In [78]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: said “my mother”
Actual Hindi Translation:  ने कहा “माँ” 
Predicted Hindi Translation:  उसकी पत्नी और सामाजिक आंदोलन का नाम इतनी क्षेत्


In [70]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: effectiveness in our lives
Actual Hindi Translation:  और सार्थकता से हमारे जीवन को भर दे 
Predicted Hindi Translation:  देश और हमारे देश में जमीन लगे बंद करने की गई 


In [71]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', Y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: in different parts of the country  different seasons are preferred for getting animals castrated
Actual Hindi Translation:  जानवर को बधिया करवाने के लिए देश के विभिन्न भागों में अलग अलग समय उपयुक़्त माना जाता है 
Predicted Hindi Translation:  ओवर या एक निर्माण की संकल्पना कप के बीच भरी 


In [72]:
#decode_sequence('What is your name?')
k+=1
(input_seq, actual_output), _ = next(train_gen)

In [73]:
input_seq

array([[   46.,    70.,  1730.,    99., 11617.,   293.,   734.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.]], dtype=float32)

In [74]:
actual_output

array([[1.000e+00, 4.160e+02, 2.280e+02, 7.700e+01, 7.000e+00, 3.572e+03,
        1.102e+03, 1.200e+02, 5.900e+01, 5.190e+02, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00]], dtype=float32)

In [79]:
model.save_weights('/content/drive/MyDrive/ColabNotebooks/prism/From1.h5')


In [84]:
# import pickle
# with open ('/content/drive/MyDrive/ColabNotebooks/prism/From1Pickle.pkl', 'wb') as file:
#     pickle.dump(model, file)